# Fine-tune NBA QA Model

This notebook fine-tunes a question-answering model on NBA-specific data.

## Overview

The fine-tuning process:
1. **Load Dataset**: Load the NBA QA dataset generated in the previous notebook
2. **Prepare Data**: Split into train/validation sets and tokenize
3. **Fine-tune Model**: Train the QA model on NBA-specific questions
4. **Evaluate**: Test the fine-tuned model's performance
5. **Save Model**: Save the fine-tuned model for use in the QA system

## Prerequisites

- Dataset generated from `1_generate_nba_qa_dataset.ipynb`
- Base model: `deepset/roberta-base-squad2`
- GPU recommended for faster training (CPU will work but slower)


## Step 0: Install Required Packages

Run this cell first to install the required packages if they're not already installed.


In [1]:
# Install required packages
# Suppress PATH warning for scripts location
%pip install transformers torch datasets accelerate ipywidgets --no-warn-script-location

print("✅ Packages installed successfully!")


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
✅ Packages installed successfully!


## Step 1: Import Required Libraries

**Note**: If you get a `ModuleNotFoundError`, make sure you ran the installation cell above.


In [2]:
import json
import os
import warnings
from pathlib import Path
from typing import Dict, List, Any
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForQuestionAnswering,
    TrainingArguments,
    Trainer,
    DefaultDataCollator
)
from transformers import pipeline

# Suppress tqdm IProgress warning
warnings.filterwarnings('ignore', category=UserWarning, module='tqdm')
warnings.filterwarnings('ignore', category=UserWarning, module='torch.utils.data.dataloader')

print("✅ Libraries imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")


✅ Libraries imported successfully!
PyTorch version: 2.8.0
CUDA available: False


## Step 2: Define the Fine-Tuner Class

This class handles:
- Loading the NBA QA dataset
- Tokenizing examples for training
- Fine-tuning the model
- Evaluating performance


In [3]:
class NBAQAFineTuner:
    """Fine-tune QA model on NBA dataset"""
    
    def __init__(self, base_model: str = "deepset/roberta-base-squad2"):
        self.base_model = base_model
        self.tokenizer = None
        self.model = None
    
    def load_dataset(self, dataset_path: str) -> Dataset:
        """Load NBA QA dataset from JSON file"""
        print(f"Loading dataset from {dataset_path}...")
        
        with open(dataset_path, 'r', encoding='utf-8') as f:
            squad_data = json.load(f)
        
        # Convert SQuAD format to list of examples
        examples = []
        for paragraph in squad_data["data"][0]["paragraphs"]:
            context = paragraph["context"]
            for qa in paragraph["qas"]:
                question = qa["question"]
                answers = qa["answers"]
                
                if answers and len(answers) > 0:
                    answer_text = answers[0]["text"]
                    answer_start = answers[0]["answer_start"]
                    
                    # Store as list of dicts (SQuAD format)
                    examples.append({
                        "context": context,
                        "question": question,
                        "answers": [{
                            "text": answer_text,
                            "answer_start": answer_start
                        }]
                    })
        
        print(f"Loaded {len(examples)} examples")
        return Dataset.from_list(examples)
    
    def prepare_dataset(self, dataset: Dataset, train_split: float = 0.8):
        """Prepare dataset for training"""
        print("Preparing dataset...")
        
        # Split into train and validation
        dataset = dataset.train_test_split(test_size=1 - train_split, seed=42)
        train_dataset = dataset["train"]
        val_dataset = dataset["test"]
        
        print(f"Train examples: {len(train_dataset)}")
        print(f"Validation examples: {len(val_dataset)}")
        
        return train_dataset, val_dataset
    
    def tokenize_function(self, examples: Dict[str, List]) -> Dict[str, Any]:
        """Tokenize examples for QA task"""
        questions = examples["question"]
        contexts = examples["context"]
        answers_list = examples["answers"]
        
        # Tokenize
        tokenized = self.tokenizer(
            questions,
            contexts,
            truncation=True,
            padding="max_length",
            max_length=512,
            return_offsets_mapping=True
        )
        
        # Find answer positions in tokenized sequence
        start_positions = []
        end_positions = []
        
        for i in range(len(questions)):
            # Extract answer from the nested structure
            if isinstance(answers_list[i], list) and len(answers_list[i]) > 0:
                answer_dict = answers_list[i][0]
                answer_text = answer_dict.get("text", "")
                answer_start_char = answer_dict.get("answer_start", -1)
            elif isinstance(answers_list[i], dict):
                answer_text = answers_list[i].get("text", "")
                answer_start_char = answers_list[i].get("answer_start", -1)
            else:
                answer_text = ""
                answer_start_char = -1
            
            # Find answer in context
            if answer_start_char >= 0:
                context_start = contexts[i].find(answer_text, max(0, answer_start_char - 10))
            else:
                context_start = contexts[i].find(answer_text)
            
            if context_start == -1 or not answer_text:
                start_positions.append(-1)
                end_positions.append(-1)
                continue
            
            context_end = context_start + len(answer_text)
            
            # Map character positions to token positions
            offset_mapping = tokenized["offset_mapping"][i]
            start_token = -1
            end_token = -1
            
            for token_idx, (token_start, token_end) in enumerate(offset_mapping):
                if token_start == 0 and token_end == 0:
                    continue
                    
                if token_start <= context_start < token_end and start_token == -1:
                    start_token = token_idx
                if token_start < context_end <= token_end and end_token == -1:
                    end_token = token_idx
                    break
            
            if start_token != -1 and end_token == -1:
                end_token = start_token + 1
            
            start_positions.append(start_token if start_token != -1 else -1)
            end_positions.append(end_token if end_token != -1 else -1)
        
        tokenized["start_positions"] = start_positions
        tokenized["end_positions"] = end_positions
        tokenized.pop("offset_mapping", None)
        
        return tokenized

print("✅ NBAQAFineTuner class defined!")


✅ NBAQAFineTuner class defined!


## Step 3: Load and Prepare Dataset

Load the dataset generated in the previous notebook.


In [4]:
# Configuration
dataset_path = "../utils/nba_qa_dataset.json"
base_model = "deepset/roberta-base-squad2"

# Check if dataset exists
if not os.path.exists(dataset_path):
    print(f"❌ Dataset not found at {dataset_path}")
    print("Please run 1_generate_nba_qa_dataset.ipynb first to generate the dataset.")
else:
    # Initialize fine-tuner
    fine_tuner = NBAQAFineTuner(base_model=base_model)
    
    # Load dataset
    dataset = fine_tuner.load_dataset(dataset_path)
    
    # Prepare train/val split
    train_dataset, val_dataset = fine_tuner.prepare_dataset(dataset, train_split=0.8)


Loading dataset from utils/nba_qa_dataset.json...
Loaded 1803 examples
Preparing dataset...
Train examples: 1442
Validation examples: 361


## Step 4: Load Model and Tokenizer

Load the base model and tokenizer. This will download the model if not already cached.


In [5]:
print("📥 Loading tokenizer and model...")
fine_tuner.tokenizer = AutoTokenizer.from_pretrained(base_model)
fine_tuner.model = AutoModelForQuestionAnswering.from_pretrained(base_model)

print(f"✅ Model loaded: {base_model}")
print(f"Model parameters: {sum(p.numel() for p in fine_tuner.model.parameters()):,}")


📥 Loading tokenizer and model...
✅ Model loaded: deepset/roberta-base-squad2
Model parameters: 124,056,578


## Step 5: Tokenize Datasets

Tokenize the training and validation datasets for model training.


In [6]:
print("🔤 Tokenizing datasets...")
train_tokenized = train_dataset.map(
    fine_tuner.tokenize_function,
    batched=True,
    remove_columns=train_dataset.column_names
)
val_tokenized = val_dataset.map(
    fine_tuner.tokenize_function,
    batched=True,
    remove_columns=val_dataset.column_names
)

# Filter out examples with invalid answer positions
train_tokenized = train_tokenized.filter(lambda x: x["start_positions"] != -1)
val_tokenized = val_tokenized.filter(lambda x: x["start_positions"] != -1)

print(f"✅ Tokenization complete!")
print(f"Train examples after filtering: {len(train_tokenized)}")
print(f"Validation examples after filtering: {len(val_tokenized)}")


🔤 Tokenizing datasets...


Map:   0%|          | 0/1442 [00:00<?, ? examples/s]

Map:   0%|          | 0/361 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1442 [00:00<?, ? examples/s]

Filter:   0%|          | 0/361 [00:00<?, ? examples/s]

✅ Tokenization complete!
Train examples after filtering: 1433
Validation examples after filtering: 358


## Step 6: Configure Training Arguments

Set up training parameters. Adjust these based on your hardware and requirements.


In [7]:
# Training configuration
output_dir = "../utils/model_nba_qa_finetuned"
num_epochs = 3
batch_size = 8
learning_rate = 2e-5
warmup_steps = 250

# Training arguments
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=learning_rate,
    warmup_steps=warmup_steps,
    weight_decay=0.01,
    logging_dir=f"{output_dir}/logs",
    logging_steps=50,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    greater_is_better=False,
    fp16=torch.cuda.is_available(),  # Use FP16 if GPU available
    report_to="none"
)

print("✅ Training arguments configured!")
print(f"Output directory: {output_dir}")
print(f"Epochs: {num_epochs}")
print(f"Batch size: {batch_size}")
print(f"Learning rate: {learning_rate}")


✅ Training arguments configured!
Output directory: utils/model_nba_qa_finetuned
Epochs: 3
Batch size: 8
Learning rate: 2e-05


## Step 7: Initialize Trainer and Start Training

This will start the fine-tuning process. Training time depends on:
- Number of examples
- Hardware (GPU vs CPU)
- Number of epochs

**Note**: This may take 15-30 minutes on GPU, or 2-4 hours on CPU.


In [8]:
# Data collator
data_collator = DefaultDataCollator()

# Trainer
trainer = Trainer(
    model=fine_tuner.model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    data_collator=data_collator,
)

# Train
print("\n🏋️ Starting training...")
print("=" * 80)
train_result = trainer.train()
print("=" * 80)

# Print training summary
print("\n✅ Training complete!")
print(f"Training loss: {train_result.training_loss:.4f}")



🏋️ Starting training...


Epoch,Training Loss,Validation Loss
1,0.162800,0.089596
2,0.179600,0.072905
3,0.083300,0.082318



✅ Training complete!
Training loss: 0.2703


## Step 8: Save the Fine-tuned Model

Save the model and tokenizer for use in the QA system.


In [9]:
print(f"💾 Saving model to {output_dir}...")
trainer.save_model()
fine_tuner.tokenizer.save_pretrained(output_dir)

print(f"✅ Model saved to: {output_dir}")
print("\nThe fine-tuned model is ready to use in the QA system!")


💾 Saving model to utils/model_nba_qa_finetuned...
✅ Model saved to: utils/model_nba_qa_finetuned

The fine-tuned model is ready to use in the QA system!


## Step 9: Evaluate the Model (Optional)

Test the fine-tuned model on the validation set to see its performance.


In [10]:
# Load fine-tuned model for evaluation
print(f"📊 Evaluating model from {output_dir}...")
qa_pipeline = pipeline(
    "question-answering",
    model=output_dir,
    tokenizer=output_dir
)

# Evaluate on validation set
correct = 0
total = 0

for example in val_dataset:
    result = qa_pipeline(
        question=example["question"],
        context=example["context"]
    )
    
    predicted_answer = result["answer"].strip()
    
    # Extract true answer
    answers = example["answers"]
    if isinstance(answers, list) and len(answers) > 0:
        true_answer = answers[0].get("text", "").strip()
    else:
        true_answer = ""
    
    # Simple exact match
    if true_answer and predicted_answer.lower() == true_answer.lower():
        correct += 1
    total += 1

accuracy = correct / total if total > 0 else 0
print(f"✅ Evaluation complete!")
print(f"Accuracy: {accuracy:.2%} ({correct}/{total})")


📊 Evaluating model from utils/model_nba_qa_finetuned...


Device set to use mps:0


✅ Evaluation complete!
Accuracy: 100.00% (361/361)


## Summary

✅ **Fine-tuning complete!**

The fine-tuned model is saved at: `utils/model_nba_qa_finetuned/`

### Next Steps

Use the fine-tuned model in the QA system:
```python
from nba_qa_system import NBAQASystem

qa_system = NBAQASystem(model_name="utils/model_nba_qa_finetuned")
result = qa_system.answer("What is LeBron James' career PPG?")
print(result.answer)
```

Or continue to the next notebook: `3_nba_qa_system.ipynb`
